# From Theory to Practice: Backtesting Our Quantile Idea

### So, Do Quantiles Actually Work?

In the last notebook, we came up with a neat idea: use quantiles to figure out a "probable" range for a candle's price movement. The logical next step is to ask the million-dollar question: can we actually make money with this?

To find out, I've used backtesting to see how different high/low percentile choices affect the win rate. Now, I understand this is a quiet, simple, and flawed representation of how the market works, but it's a starting point. And as we'll see, even simple models can show some interesting data. Let's build a strategy using the `backtesting.py` library and see if our theory holds any water.

In [ ]:
# First, let's get the necessary tools for the job.
%pip install backtesting yfinance pandas

### Building the strategy

Here's the game plan. We'll create a strategy that does the following for every candle:
1.  **Look Back:** It will look at all the *past* candles to calculate the percentage price moves (just like we did manually).
2.  **Calculate Quantiles:** Based on that history, it will calculate a quantile value (e.g., the 50th percentile) for both upward and downward moves. This gives us our dynamic thresholds.
3.  **Set TP and SL:** It will place a new trade using the current price and set a Take Profit (TP) and Stop Loss (SL) based on those quantile thresholds.

The beauty is that the TP and SL are no longer fixed numbers; they adapt as the bot sees more and more data. Let's code it up.

In [ ]:
import yfinance as yf
import pandas as pd
from backtesting import Strategy, Backtest
 
# This is our lovely trading bot, QuantileStrategy.
class QuantileStrategy(Strategy):  
    # We can tell the bot which quantile to use for TP and SL.
    h_percentile = None # Our Take Profit quantile
    l_percentile = None # Our Stop Loss quantile
    mode = "long" # Let's just focus on buying for now

    # This function defines HOW we calculate the quantile. 
    # 'expanding' means it uses all history from the start.
    @staticmethod
    def default_func(series: pd.Series, q: float) -> pd.Series:
        return series.expanding(min_periods=10).quantile(q)

    func = default_func  # We can override this later for more complex ideas.

    # The 'init' method is the bot's morning coffee. It runs once at the start to set everything up.
    def init(self):
        self.mode_coef = 1 if self.mode == "long" else -1
        self.mode_trade = self.buy if self.mode == "long" else self.sell

        o = self.data.Open
        h = self.data.High
        l = self.data.Low

        # Calculate the relative intrabar moves, just like in the last notebook.
        # .shift(1) is CRUCIAL. It prevents us from using the current candle's data to make a decision (no cheating!).
        h_move = pd.Series((h - o) / o).shift(1) 
        l_move = pd.Series((o - l) / o).shift(1) 
        
        # Here, we calculate our dynamic TP and SL thresholds for the entire dataset.
        self.h_threshold = self.I(self.func, h_move, self.h_percentile) if self.h_percentile is not None else None
        self.l_threshold = self.I(self.func, l_move, self.l_percentile) if self.l_percentile is not None else None

    # The 'next' method is the main loop. It runs for every candle and asks, "What do we do now?"
    def next(self):
        # Wait until we have enough data to make a meaningful calculation.
        if len(self.data) < 10:
            return

        price = self.data.Close # Use the last closing price as our entry point.
        h = self.h_threshold # Get the most recent quantile value.
        l = self.l_threshold

        if pd.isna(h) and pd.isna(l):
            return

        # Calculate the actual TP and SL prices.
        tp = price * (1 + h * self.mode_coef) if h is not None else None
        sl = price * (1 - l * self.mode_coef) if l is not None else None

        # Close any open trade before starting a new one.
        self.position.close()
        
        # Execute the trade! Let's see what happens.
        self.mode_trade(tp=tp, sl=sl, size=1)

### The Moment of Truth

Alright, the bot is coded. It's time to download some fresh data, plug in our strategy, and see how much theoretical money we made (or, more likely, lost). We'll start by setting our Stop Loss at the 50th percentile, meaning we're placing it at the *median* down move. For now, we'll leave the Take Profit open.

In [ ]:
# Let's grab some data for Gold again.
tickers = ["GC=F"]
period = "1mo"
interval = "1h"

print("Fetching data for the grand experiment...")
data = yf.download(tickers=tickers, period=period, interval=interval, multi_level_index=False)

# Initialize the backtest environment.
bt = Backtest(data, QuantileStrategy, cash=10000, commission=.002)

# Run the backtest. We'll set the stop-loss percentile to 0.5 (the 50% quantile).
result = bt.run(h_percentile=0.5)

print("\n--- Backtest Results ---")
print(result)

### The Verdict and What's Next

So, there are the results. Genius strategy or a spectacular failure? The numbers speak for themselves. 

This simple backtest is made as a foundation for higher research and testing. 
We can now build upon this strategy in many ways:

*   **Different Calculation Methods:** Instead of an `expanding` window, we could use a `rolling` window to only consider the last N candles.
*   **Smarter Filtering:** We could filter the historical data to only calculate quantiles during high-volatility periods, for example.
*   **Optimization:** We can run this backtest hundreds of times to find the *optimal* quantile percentages for TP and SL. (good foreshadowing)

I would not consider this perfect, but it seems like an okay start. We've successfully turned a statistical idea into a testable hypothesis, which is a step forward.